# Information

The folowing notebook is a completely manual setup of a _great_expectations_ pipeline. Usually many of the steps (especially setup) are fulfilled by the _great_expectations_ clientl. This notebook is only for educational reasons to get a deeper understanding on what is going on within the library.

# Setup

In [12]:
import json
import great_expectations as ge
import pandas as pd
from ruamel import yaml
from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest


## Create datasource config and load 

In [28]:
# Load DataContext to memory
context = ge.get_context()

In [16]:
datasource_yaml="""
name: transactions_use_case
class_name: Datasource
execution_engine:
  class_name: PandasExecutionEngine
data_connectors:
  default_inferred_data_connector_name:
    class_name: InferredAssetFilesystemDataConnector
    base_directory: ../data
    default_regex:
      group_names: 
        - data_asset_name
      pattern: (.*)
  default_runtime_data_connector_name:
    class_name: RuntimeDataConnector
    batch_identifiers:
      - default_identifier_name
"""

In [17]:
context.test_yaml_config(datasource_yaml) 

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: PandasExecutionEngine
Data Connectors:
	default_inferred_data_connector_name : InferredAssetFilesystemDataConnector

	Available data_asset_names (3 of 3):
		articles.csv (1 of 1): ['articles.csv']
		transactions.csv (1 of 1): ['transactions.csv']
		transactions_faulty.csv (1 of 1): ['transactions_faulty.csv']

	Unmatched data_references (0 of 0):[]

	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



After succesfully setting up the config and testing it, we have to load it to the appropriate palace in oure _great_expectations.yaml_ file wich has been created with the initialisation of great_expectations

In [19]:
context.add_datasource(**yaml.load(datasource_yaml))

/tmp/ipykernel_1180/2045274942.py:1: UnsafeLoaderWarning: 
The default 'Loader' for 'load(stream)' without further arguments can be unsafe.
Use 'load(stream, Loader=ruamel.yaml.Loader)' explicitly if that is OK.
Alternatively include the following in your code:

  import warnings
  warnings.simplefilter('ignore', ruamel.yaml.error.UnsafeLoaderWarning)

In most other cases you should consider using 'safe_load(stream)'
  context.add_datasource(**yaml.load(datasource_yaml))


## Testing the new datasource

In [38]:
batch_request = RuntimeBatchRequest(
    datasource_name="transactions_use_case", # this has to be equal as the defined data_source_name in the datasource_yaml
    data_connector_name="default_runtime_data_connector_name",
    data_asset_name="transactions",  # This can be anything that identifies this data_asset for you
    runtime_parameters={"path": "./data/transactions.csv"},  # Add your path here.
    batch_identifiers={"default_identifier_name": "default_identifier"},
)

In [39]:
context.create_expectation_suite(
    expectation_suite_name="test_suite", overwrite_existing=True
)

{
  "data_asset_type": null,
  "expectation_suite_name": "test_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "meta": {
    "great_expectations_version": "0.14.6"
  }
}

In [40]:
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="test_suite"
)
print(validator.head())

         date article_id  value
0  05.07.2021      200XL  49.99
1  20.06.2021       400S  19.99
2  15.04.2021      200XL  49.99
3  02.04.2021       500M  79.99
4  02.04.2021       100L  29.99


# Creating an expectation suite

To create a testing pipeline with set of expectations a suite has to be created. It is a JSON with all expectations that will be run on a given dataset.

There are 2 options to setup expectations.
* Use the profiler and a "healthy" dataset to auto generate expectations based on the input tables form and simple metrics (mean, min, max) 
* Create an expectations JSON manually by utilizing already implemented expectations and/or create custom expectations to test against certain input data
   
This notebook will go with the latter approach. To use the profiler just run _great_expectations suite new_ and select the profiler option. This will start a new helper notebook where expectatins based on the input data are created. 

In [62]:
from great_expectations.core.expectation_configuration import ExpectationConfiguration
from great_expectations.data_context.types.resource_identifiers import ExpectationSuiteIdentifier

### 1 Create expectation suite

In [89]:
context.create_expectation_suite(
    expectation_suite_name="transaction_validation_suite", overwrite_existing=True
)

{
  "data_asset_type": null,
  "expectation_suite_name": "transaction_validation_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "meta": {
    "great_expectations_version": "0.14.6"
  }
}

### 2 Set suite to the new expectation suite

In [90]:
try:
    suite = context.get_expectation_suite(expectation_suite_name=expectation_suite_name)
    print(
        f'Loaded ExpectationSuite "{suite.expectation_suite_name}" containing {len(suite.expectations)} expectations.'
    )
except DataContextError:
    suite = context.create_expectation_suite(
        expectation_suite_name=expectation_suite_name
    )
    print(f'Created ExpectationSuite "{suite.expectation_suite_name}".')

Loaded ExpectationSuite "transaction_validation_suite" containing 0 expectations.


### 3 Create expectations

In [91]:
# Create an Expectation
articles = pd.read_csv(("./data/articles.csv"))
print(articles.head())

article_id_list= articles['article_id'].values.tolist()

expectation_configuration = ExpectationConfiguration(
   expectation_type="expect_column_values_to_be_in_set",
   kwargs={
      "column":"article_id",
      "value_set": article_id_list
   },
   meta={
       "notes": {
         "content": """The input value set is derived from the articles.csv. This way the validation data will always be compared 
         to the currently available articles. Optionally another suite for checking articles may be beneficial"""
      }
   }
)
# Add the Expectation to the suite
suite.add_expectation(expectation_configuration=expectation_configuration)

  article_id article_name  article_color_id article_size_id
0       100L      T-Shirt               100               L
1      200XL        Pants               200              XL
2       300M        Jeans               300              XM
3       400S          Hat               400               S
4       500M     Sneakers               500               M


{"expectation_type": "expect_column_values_to_be_in_set", "kwargs": {"column": "article_id", "value_set": ["100L", "200XL", "300M", "400S", "500M"]}, "meta": {"notes": {"content": "The input value set is derived from the articles.csv. This way the validation data will always be compared \n         to the currently available articles. Optionally another suite for checking articles may be beneficial"}}}

In [92]:
 expectation_configuration = ExpectationConfiguration(
    expectation_type= "expect_column_values_to_not_be_null",
      kwargs={
        "column": "value"
    }
 )
suite.add_expectation(expectation_configuration=expectation_configuration)

{"expectation_type": "expect_column_values_to_not_be_null", "kwargs": {"column": "value"}, "meta": {}}

In [93]:
expectation_configuration = ExpectationConfiguration(
    expectation_type= "expect_column_values_to_not_be_null",
      kwargs={
        "column": "article_id"
      }
)
suite.add_expectation(expectation_configuration=expectation_configuration)

{"expectation_type": "expect_column_values_to_not_be_null", "kwargs": {"column": "article_id"}, "meta": {}}

### 4 Save created expectations to expectation suite JSON

In [94]:
print(context.get_expectation_suite(expectation_suite_name=expectation_suite_name))
context.save_expectation_suite(expectation_suite=suite, expectation_suite_name=expectation_suite_name)


{
  "data_asset_type": null,
  "expectation_suite_name": "transaction_validation_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "meta": {
    "great_expectations_version": "0.14.6"
  }
}


'/home/kevinkrs/programming/GreatExpectationsV2/great_expectations/expectations/transaction_validation_suite.json'

In [127]:
suite_identifier = ExpectationSuiteIdentifier(expectation_suite_name=expectation_suite_name)
context.build_data_docs(resource_identifiers=[suite_identifier])
# context.open_data_docs(resource_identifier=suite_identifier)

Unrecognized Expectation suite notes format. Skipping rendering.
An unexpected Exception occurred during data docs rendering.  Because of this error, certain parts of data docs will not be rendered properly and/or may not appear altogether.  Please use the trace, included in this message, to diagnose and repair the underlying issue.  Detailed information follows:
                TypeError: "'NoneType' object is not iterable".  Traceback: "Traceback (most recent call last):
  File "/home/kevinkrs/programming/GreatExpectationsV2/venv/lib/python3.8/site-packages/great_expectations/render/renderer/site_builder.py", line 473, in build
    viewable_content = self.view_class.render(
  File "/home/kevinkrs/programming/GreatExpectationsV2/venv/lib/python3.8/site-packages/great_expectations/render/view/view.py", line 103, in render
    document = document.to_json_dict()
  File "/home/kevinkrs/programming/GreatExpectationsV2/venv/lib/python3.8/site-packages/great_expectations/render/types/__init_

{'local_site': 'file:///home/kevinkrs/programming/GreatExpectationsV2/great_expectations/uncommitted/data_docs/local_site/index.html'}

# Validation

## Create a checkpoint yaml 

In [115]:
from pprint import pprint
from ruamel.yaml import YAML

yaml = YAML()

### 1 Check availabl data and suites

In [116]:
# check on available assets
pprint(context.get_available_data_asset_names())

{'transactions_use_case': {'default_inferred_data_connector_name': ['articles.csv',
                                                                    'transactions.csv',
                                                                    'transactions_faulty.csv'],
                           'default_runtime_data_connector_name': ['transactions']}}


In [117]:
# check available suites (expectation JSONs) 
context.list_expectation_suite_names()

['transaction_validation_suite']

### 2 Create checkpoint yaml 

In [118]:
# for an easy checkpoint creation use the great_expectations client
my_checkpoint_name = (
    "transaction_validation"  
)

data_to_be_validated_name = "transactions_faulty.csv" # parametrization of the to-be-validated file 

yaml_config = f"""
name: {my_checkpoint_name}
config_version: 1.0
class_name: SimpleCheckpoint
run_name_template: "%Y%m%d-%H%M%S-my-run-name-template"
validations:
  - batch_request:
      datasource_name: transactions_use_case
      data_connector_name: default_inferred_data_connector_name
      data_asset_name: {data_to_be_validated_name}
      data_connector_query:
        index: -1
    expectation_suite_name: transaction_validation_suite
"""
print(yaml_config)


name: transaction_validation
config_version: 1.0
class_name: SimpleCheckpoint
run_name_template: "%Y%m%d-%H%M%S-my-run-name-template"
validations:
  - batch_request:
      datasource_name: transactions_use_case
      data_connector_name: default_inferred_data_connector_name
      data_asset_name: transactions_faulty.csv
      data_connector_query:
        index: -1
    expectation_suite_name: transaction_validation_suite



### 3 Test checkpoint

In [119]:
my_checkpoint = context.test_yaml_config(yaml_config=yaml_config)

Attempting to instantiate class from config...
	Instantiating as a SimpleCheckpoint, since class_name is SimpleCheckpoint
	Successfully instantiated SimpleCheckpoint


Checkpoint class name: SimpleCheckpoint


In [120]:
print(my_checkpoint.get_config(mode="yaml"))

name: transaction_validation
config_version: 1.0
template_name:
module_name: great_expectations.checkpoint
class_name: Checkpoint
run_name_template: '%Y%m%d-%H%M%S-my-run-name-template'
expectation_suite_name:
batch_request: {}
action_list:
  - name: store_validation_result
    action:
      class_name: StoreValidationResultAction
  - name: store_evaluation_params
    action:
      class_name: StoreEvaluationParametersAction
  - name: update_data_docs
    action:
      class_name: UpdateDataDocsAction
      site_names: []
evaluation_parameters: {}
runtime_configuration: {}
validations:
  - batch_request:
      datasource_name: transactions_use_case
      data_connector_name: default_inferred_data_connector_name
      data_asset_name: transactions_faulty.csv
      data_connector_query:
        index: -1
    expectation_suite_name: transaction_validation_suite
profilers: []
ge_cloud_id:
expectation_suite_ge_cloud_id:



### 4 Load checkpoint to great_expectations environment

In [121]:
context.add_checkpoint(**yaml.load(yaml_config))

{
  "action_list": [
    {
      "name": "store_validation_result",
      "action": {
        "class_name": "StoreValidationResultAction"
      }
    },
    {
      "name": "store_evaluation_params",
      "action": {
        "class_name": "StoreEvaluationParametersAction"
      }
    },
    {
      "name": "update_data_docs",
      "action": {
        "class_name": "UpdateDataDocsAction",
        "site_names": []
      }
    }
  ],
  "batch_request": {},
  "class_name": "Checkpoint",
  "config_version": 1.0,
  "evaluation_parameters": {},
  "module_name": "great_expectations.checkpoint",
  "name": "transaction_validation",
  "profilers": [],
  "run_name_template": "%Y%m%d-%H%M%S-my-run-name-template",
  "runtime_configuration": {},
  "validations": [
    {
      "batch_request": {
        "datasource_name": "transactions_use_case",
        "data_connector_name": "default_inferred_data_connector_name",
        "data_asset_name": "transactions_faulty.csv",
        "data_connector_query"

## Run Validation

In [128]:
context.run_checkpoint(checkpoint_name=my_checkpoint_name)

Calculating Metrics:   0%|          | 0/12 [00:00<?, ?it/s]

Unrecognized Expectation suite notes format. Skipping rendering.
An unexpected Exception occurred during data docs rendering.  Because of this error, certain parts of data docs will not be rendered properly and/or may not appear altogether.  Please use the trace, included in this message, to diagnose and repair the underlying issue.  Detailed information follows:
                TypeError: "'NoneType' object is not iterable".  Traceback: "Traceback (most recent call last):
  File "/home/kevinkrs/programming/GreatExpectationsV2/venv/lib/python3.8/site-packages/great_expectations/render/renderer/site_builder.py", line 473, in build
    viewable_content = self.view_class.render(
  File "/home/kevinkrs/programming/GreatExpectationsV2/venv/lib/python3.8/site-packages/great_expectations/render/view/view.py", line 103, in render
    document = document.to_json_dict()
  File "/home/kevinkrs/programming/GreatExpectationsV2/venv/lib/python3.8/site-packages/great_expectations/render/types/__init_

{
  "run_id": {
    "run_time": "2022-02-18T11:47:44.804777+00:00",
    "run_name": "20220218-114744-my-run-name-template"
  },
  "run_results": {
    "ValidationResultIdentifier::transaction_validation_suite/20220218-114744-my-run-name-template/20220218T114744.804777Z/6bb28be6367073d63550b8552baa3682": {
      "validation_result": {
        "success": false,
        "results": [
          {
            "exception_info": {
              "raised_exception": false,
              "exception_traceback": null,
              "exception_message": null
            },
            "success": false,
            "meta": {},
            "expectation_config": {
              "expectation_type": "expect_column_values_to_be_in_set",
              "kwargs": {
                "column": "article_id",
                "value_set": [
                  "100L",
                  "200XL",
                  "300M",
                  "400S",
                  "500M"
                ],
                "batch_id":

## Single expectations validation with python

### Import the required data for check

In [97]:
transactions_df = ge.read_csv("./data/transactions.csv")
transactions_faulty_df = ge.read_csv("./data/transactions_faulty.csv")

# create set of all article id's
articles = pd.read_csv(("./data/articles.csv"))
print(articles.head())

article_id= articles['article_id'].values.tolist()
print(article_id)


  article_id article_name  article_color_id article_size_id
0       100L      T-Shirt               100               L
1      200XL        Pants               200              XL
2       300M        Jeans               300              XM
3       400S          Hat               400               S
4       500M     Sneakers               500               M
['100L', '200XL', '300M', '400S', '500M']


In [99]:
print(transactions_df.expect_column_values_to_be_in_set('article_id', article_id))

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": true,
  "meta": {},
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "column": "article_id",
      "value_set": [
        "100L",
        "200XL",
        "300M",
        "400S",
        "500M"
      ],
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "result": {
    "element_count": 6,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  }
}


In [98]:
print(transactions_faulty_df.expect_column_values_to_be_in_set('article_id', article_id))

{
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "success": false,
  "meta": {},
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "column": "article_id",
      "value_set": [
        "100L",
        "200XL",
        "300M",
        "400S",
        "500M"
      ],
      "result_format": "BASIC"
    },
    "meta": {}
  },
  "result": {
    "element_count": 10,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 3,
    "unexpected_percent": 30.0,
    "unexpected_percent_total": 30.0,
    "unexpected_percent_nonmissing": 30.0,
    "partial_unexpected_list": [
      "100KL",
      "800S",
      "800S"
    ]
  }
}
